In [2]:
import numpy as np
from keras import backend as keras
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
import tensorflow as tf
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2

from sklearn.model_selection import train_test_split

from glob import glob
import cv2

from PIL import Image

import os

from matplotlib import pyplot as plt

import tensorflow.keras.backend as K

2023-12-20 21:53:26.688881: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-20 21:53:26.766355: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-20 21:53:26.767817: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-20 21:53:37.219981: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
os.chdir("drive/MyDrive/BioCV2023")

In [3]:
# code from https://medium.com/the-owl/weighted-categorical-cross-entropy-loss-in-keras-edaee1df44ee
import custom_loss_function as clf
from keras.saving import *
import tensorflow.keras as keras
import warnings

@tf.keras.saving.register_keras_serializable(name="weighted_categorical_crossentropy")
def weighted_categorical_crossentropy(target, output, weights, axis=-1):
    target = tf.convert_to_tensor(target)
    output = tf.convert_to_tensor(output)
    target.shape.assert_is_compatible_with(output.shape)
    weights = tf.reshape(tf.convert_to_tensor(weights, dtype=target.dtype), (1,-1))

    # Adjust the predictions so that the probability of
    # each class for every sample adds up to 1
    # This is needed to ensure that the cross entropy is
    # computed correctly.
    output = output / tf.reduce_sum(output, axis, True)

    # Compute cross entropy from probabilities.
    epsilon_ = tf.constant(tf.keras.backend.epsilon(), output.dtype.base_dtype)
    output = tf.clip_by_value(output, epsilon_, 1.0 - epsilon_)     # avoiding extreme values (0 or 1)
    return -tf.reduce_sum(weights * target * tf.math.log(output), axis=axis)    # reduced sum so there is 1 loss value per pixel


@tf.keras.saving.register_keras_serializable(name="WeightedCategoricalCrossentropy")
class WeightedCategoricalCrossentropy:
    def __init__(
        self,
        weights,
        label_smoothing=0.0,
        axis=-1,
        name="weighted_categorical_crossentropy",
        fn = None,
    ):
        """Initializes `WeightedCategoricalCrossentropy` instance.
        Args:
          from_logits: Whether to interpret `y_pred` as a tensor of
            [logit](https://en.wikipedia.org/wiki/Logit) values. By default, we
            assume that `y_pred` contains probabilities (i.e., values in [0,
            1]).
          label_smoothing: Float in [0, 1]. When 0, no smoothing occurs. When >
            0, we compute the loss between the predicted labels and a smoothed
            version of the true labels, where the smoothing squeezes the labels
            towards 0.5.  Larger values of `label_smoothing` correspond to
            heavier smoothing.
          axis: The axis along which to compute crossentropy (the features
            axis).  Defaults to -1.
          name: Name for the op. Defaults to 'weighted_categorical_crossentropy'.
        """
        super().__init__()
        self.weights = weights # tf.reshape(tf.convert_to_tensor(weights),(1,-1))
        self.label_smoothing = label_smoothing
        self.name = name
        self.fn = weighted_categorical_crossentropy if fn is None else fn

    def __call__(self, y_true, y_pred, axis=-1):
        if isinstance(axis, bool):
            raise ValueError(
                "`axis` must be of type `int`. "
                f"Received: axis={axis} of type {type(axis)}"
            )
        y_pred = tf.convert_to_tensor(y_pred)
        y_true = tf.cast(y_true, y_pred.dtype)
        self.label_smoothing = tf.convert_to_tensor(self.label_smoothing, dtype=y_pred.dtype)

        if y_pred.shape[-1] == 1:
            warnings.warn(
                "In loss categorical_crossentropy, expected "
                "y_pred.shape to be (batch_size, num_classes) "
                f"with num_classes > 1. Received: y_pred.shape={y_pred.shape}. "
                "Consider using 'binary_crossentropy' if you only have 2 classes.",
                SyntaxWarning,
                stacklevel=2,
            )

        def _smooth_labels():
            num_classes = tf.cast(tf.shape(y_true)[-1], y_pred.dtype)
            return y_true * (1.0 - self.label_smoothing) + (self.label_smoothing / num_classes)

        y_true = tf.__internal__.smart_cond.smart_cond(self.label_smoothing, _smooth_labels, lambda: y_true)

        return tf.reduce_mean(self.fn(y_true, y_pred, self.weights, axis=axis))

    def get_config(self):
        config = {"name":self.name, "weights": self.weights, "fn": weighted_categorical_crossentropy}

        # base_config = super().get_config()
        return dict(list(config.items()))

    @classmethod
    def from_config(cls, config):
        """Instantiates a `Loss` from its config (output of `get_config()`).
        Args:
            config: Output of `get_config()`.
        """
        return cls(**config)


### Hyperparameters

In [4]:
IMAGE_SIZE = 256
num_classes = 34    # now I have 34 classes since I'm detecting all possible borders
batch = 4
LR = 1e-4
EPOCHS = 20
NUM_IMAGES = 522

### Auxiliar functions
Here are implemented functions for creating the dataset
* loading paths
* division of training/validation/test sets
* loading and reshaping (+ normalizing) images and masks
* data augmentation

In [5]:
image_path = "./Preprocessed_Set/T1DUAL/OutPhase/Images/" 
regex_images_paths = os.path.join(image_path, "*.png")
mask_path = "./Preprocessed_Set/T1DUAL/OutPhase/Masks/"   
regex_masks_paths = os.path.join(mask_path, "*.png")
checkpoint_path = "./trained_models/model4/96.63cotp-96.54cott/"
#  80% training, 10% validation, 10% test
def load_data():
  images_paths = sorted(glob(regex_images_paths))
  masks_paths = sorted(glob(regex_masks_paths))
  train_x, tmp_x, train_y, tmp_y = train_test_split(images_paths, masks_paths, test_size=0.2, random_state=42, shuffle=True)
  valid_x, test_x, valid_y, test_y = train_test_split(tmp_x, tmp_y, test_size=0.5, random_state=42, shuffle=True)

  return (train_x, train_y), (valid_x, valid_y), (test_x, test_y)


In [6]:
# loading and normalizing to range 0-1 an image
def read_image(path):
    path = path.decode()
    x = np.array(Image.open(path))
    x = cv2.resize(x, (IMAGE_SIZE, IMAGE_SIZE), interpolation=0)
    #x = x.astype(float)/255.0
    x = x/255.0
    return x

# loading the mask
def read_mask(path):
    path = path.decode()
    x = np.array(Image.open(path))
    x = cv2.resize(x, (IMAGE_SIZE, IMAGE_SIZE), interpolation=0)
    x = K.cast(K.one_hot(x, num_classes=num_classes), tf.uint8)
    #x = np.expand_dims(x, axis=-1) 
    return x

# loading a image and mask
def tf_parse(x, y):
    def _parse(x, y):
        x = read_image(x)
        y = read_mask(y)
        return x, y

    x, y = tf.numpy_function(_parse, [x, y], [tf.float64, tf.uint8])
    x.set_shape([IMAGE_SIZE, IMAGE_SIZE])
    y.set_shape([IMAGE_SIZE, IMAGE_SIZE, num_classes])
    return x, y

# creating training dataset
def tf_dataset_train(x, y, batch=4):
    dataset = tf.data.Dataset.from_tensor_slices((x, y))    # creating the dataset with the paths...
    dataset = dataset.shuffle(buffer_size=500)
    dataset = dataset.repeat()
    dataset = dataset.map(tf_parse, num_parallel_calls=tf.data.AUTOTUNE)    # ... and resolving the paths
    dataset = dataset.batch(batch)
    #dataset = dataset.map(lambda x, y: (data_augmentation(x, training=True), y), num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.prefetch(1)       # per caricare in anticipo anche 1 elemento dopo
    return dataset

def tf_dataset_valid(x, y, batch=batch):
    dataset = tf.data.Dataset.from_tensor_slices((x, y))
    dataset = dataset.shuffle(buffer_size=50)
    dataset = dataset.repeat()
    dataset = dataset.map(tf_parse, num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.batch(batch)
    dataset = dataset.prefetch(1)
    return dataset


### U-Net model
Here U-Net model is implemented. Notice that:
* CNN output matrix shape is `(row_img, col_img, num_classes)`, where for each pixel _p_ for each class _i_ a probability of belonging to class _i_ is assigned to pixel (through softmax) 

In [7]:
def recall(y_true, y_pred):
  true_positives = K.sum(K.round(K.clip(y_true[:,:,2:] * y_pred[:,:,2:], 0, 1)))
  possible_positives = K.sum(K.round(K.clip(y_true[:,:,2:], 0, 1)))
  recall_keras = true_positives / (possible_positives + K.epsilon())
  return recall_keras


def precision(y_true, y_pred):
  true_positives = K.sum(K.round(K.clip(y_true[:,:,2:] * y_pred[:,:,2:], 0, 1)))
  predicted_positives = K.sum(K.round(K.clip(y_pred[:,:,2:], 0, 1)))
  precision_keras = true_positives / (predicted_positives + K.epsilon())
  return precision_keras

def correct_over_total_pred(y_true, y_pred):
  correct_prediction = K.sum(K.round(K.clip(y_true[:,:,2:] * y_pred[:,:,2:], 0, 1)))
  total_prediction = K.sum(K.round(K.clip(y_pred[:,:,2:], 0, 1),))
  result = correct_prediction / (total_prediction + K.epsilon())
  return result

def cotp(y_true, y_pred):
  return correct_over_total_pred(y_true, y_pred)

def wrong_over_total_pred(y_true, y_pred):
  return 1 - correct_over_total_pred(y_true, y_pred)

def correct_over_total_true(y_true, y_pred):
  true_positives = K.sum(K.round(K.clip(y_true[:,:,2:] * y_pred[:,:,2:], 0, 1)))
  possible_positives = K.sum(K.round(K.clip(y_true[:,:,2:], 0, 1)))
  recall_keras = true_positives / (possible_positives + K.epsilon())
  return recall_keras

def cott(y_true, y_pred):
  return correct_over_total_true(y_true, y_pred)

def true_over_total_pred(y_true, y_pred):
  true_positives = K.sum(K.round(K.clip(y_true[:,:,2:] * y_pred[:,:,2:], 0, 1)))
  predicted_positives = K.sum(K.round(K.clip(y_pred[:,:,2:], 0, 1)))
  precision_keras = true_positives / (predicted_positives + K.epsilon())
  return precision_keras


In [10]:
skip_connection_names = ["input_image", "conv1_conv", "conv2_block3_1_relu", "conv3_block4_1_relu", "conv4_block23_1_relu"]

def unet(input_size = (256,256, 1)):
    inputs = Input(input_size, name="input_image")
    inputs0 = tf.image.grayscale_to_rgb(inputs)  # converting images to RGB since ResNet only receives RGB images
    base_model = tf.keras.applications.ResNet101V2(input_tensor=inputs0,include_top=False,weights='imagenet')
    base_model.trainable = False

    base_model_output = base_model.get_layer("post_relu").output

    x_skip_1 = base_model.get_layer(skip_connection_names[-1]).output
    x_skip_2 = base_model.get_layer(skip_connection_names[-2]).output
    x_skip_3 = base_model.get_layer(skip_connection_names[-3]).output
    x_skip_4 = base_model.get_layer(skip_connection_names[-4]).output
    x_skip_5 = base_model.get_layer(skip_connection_names[-5]).output
    drop1 = Dropout(0.5)(base_model_output)

    up6 = Conv2D(512, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(drop1))
    merge6 = concatenate([x_skip_1,up6], axis = 3)   # this is the where the skip connection arrives from the respective encoder layer
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)

    up7 = Conv2D(256, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv6))
    merge7 = concatenate([x_skip_2,up7], axis = 3)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)

    up8 = Conv2D(128, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv7))
    merge8 = concatenate([x_skip_3,up8], axis = 3)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)

    up9 = Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv8))
    merge9 = concatenate([x_skip_4,up9], axis = 3)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge9)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)

    up10 = Conv2D(32, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv9))
    merge10 = concatenate([x_skip_5,up10], axis = 3)
    conv10 = Conv2D(32, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge10)
    conv10 = Conv2D(32, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv10)
    conv10 = Conv2D(num_classes, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv10)
    conv11 = Conv2D(num_classes, 1, activation = 'softmax')(conv10)

    model = Model(inputs = inputs, outputs = conv11)

    weights = [1, 3, 9, 14, 14, 14]
    border_weights = [18] * (33-5)
    weights = weights + border_weights

    model.compile(optimizer = Adam(learning_rate = 1e-4), loss = clf.WeightedCategoricalCrossentropy(weights), metrics = ['accuracy', cotp, cott])   # posso usare metriche implementate da me (callbacks)


    return model

## Model instanciation and training

### Dataset creation

In [8]:
(train_x, train_y), (valid_x, valid_y), (test_x, test_y) = load_data()

train_dataset = tf_dataset_train(train_x, train_y)
valid_dataset = tf_dataset_valid(valid_x, valid_y)
test_dataset = tf_dataset_valid(test_x, test_y)

In [52]:
model = unet()

model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    save_weights_only=False,
    monitor='val_accuracy',
    mode='auto',
    save_best_only=True)

#model.summary()
#tf.keras.utils.plot_model(model, show_shapes=True)

In [53]:
# Commencement of training
train_steps = len(train_x)//batch
valid_steps = len(valid_x)//batch

if len(train_x) % batch != 0:   ## equivalent of doing ceil in the code before
    train_steps += 1
if len(valid_x) % batch != 0:
    valid_steps += 1

model.fit(
    train_dataset,
    validation_data = valid_dataset,
    epochs=EPOCHS,
    steps_per_epoch=train_steps,
    validation_steps=valid_steps,
    callbacks=[model_checkpoint_callback]
)

Epoch 1/20
105/105 [==============================] - 72s 588ms/step - loss: 4.1705 - accuracy: 0.7293 - cotp: 0.8455 - cott: 0.5773 - val_loss: 2.9842 - val_accuracy: 0.9000 - val_cotp: 0.9748 - val_cott: 0.7553
Epoch 2/20
105/105 [==============================] - 56s 535ms/step - loss: 2.1259 - accuracy: 0.9141 - cotp: 0.9553 - cott: 0.8654 - val_loss: 2.2803 - val_accuracy: 0.9096 - val_cotp: 0.9530 - val_cott: 0.8836
Epoch 3/20
105/105 [==============================] - 60s 575ms/step - loss: 1.7267 - accuracy: 0.9202 - cotp: 0.9582 - cott: 0.8879 - val_loss: 1.6869 - val_accuracy: 0.9123 - val_cotp: 0.9623 - val_cott: 0.8766
Epoch 4/20
105/105 [==============================] - 57s 550ms/step - loss: 1.3218 - accuracy: 0.9272 - cotp: 0.9582 - cott: 0.9053 - val_loss: 1.3942 - val_accuracy: 0.9347 - val_cotp: 0.9689 - val_cott: 0.9051
Epoch 5/20
105/105 [==============================] - 57s 547ms/step - loss: 1.0769 - accuracy: 0.9352 - cotp: 0.9586 - cott: 0.9181 - val_loss: 1.1

## Postprocessing: border mapped to original classes and image plotting

In [9]:
# used to detect borders in preprocessing and now for mapping borders to the 6 original classes
border_type = {
    0: [0,0, 6, 7, 8, 9],   # background border with (by index) 0) background, 1) body, 2) liver, 3) right kidney, 4) left kidney, 5) spleen
    1: [0,0,10,11,12,13],   # body border with ... (same)
    2: [14,15,0,16,17,18],  # liver border with ... (same)
    3: [19,20,21,0,22,23],  # right kidney border with ... (same)
    4: [24,25,26,27,0,28],  # left kidney border with ... (same)
    5: [29,30,31,32,33,0]   # spleen border with ... (same)
}

# creating a map for mapping borders to the 6 original classes, by "inverting" border_type map
to_real_class = dict([[0,0],[1,1],[2,2],[3,3],[4,4],[5,5]])
for (key,values) in border_type.items():
    for value in values:
        if value == 0: continue
        to_real_class[value] = key

In [10]:
def mask_from_prob(mask):
    def _mask_from_prob(mask):
        mask = tf.argmax(mask, axis=-1)
        return mask

    msk = _mask_from_prob(mask)
    msk = np.array(msk)
    for row in range(msk.shape[0]):
        for col in range(msk.shape[1]):
            msk[row][col] = to_real_class[msk[row][col]]
    return msk

def display_sample(display_list):
    plt.figure(figsize=(16, 16))
    title = ['Input Image', 'True Mask', 'Predicted Mask']
    for i in range(len(display_list)):
        plt.subplot(1, len(display_list), i+1)
        plt.title(title[i])
        plt.imshow(tf.keras.preprocessing.image.array_to_img(display_list[i]))
        plt.axis('off')
    plt.show()

def show_predictions(i=0):
    prediction = model.predict(sample_image)
    true_mask = mask_from_prob(sample_mask[i])
    pred_mask = mask_from_prob(prediction[i])
    plt.subplot(1, 3, 1)
    plt.imshow(sample_image[i], cmap="gray")
    plt.subplot(1, 3, 2)
    plt.imshow(true_mask, cmap="gray")
    plt.subplot(1, 3, 3)
    plt.imshow(pred_mask, cmap="gray")


In [ ]:
for image, mask in test_dataset.take(1):
    sample_image, sample_mask = image, mask

prediction = model.predict(sample_image)
plt.figure(figsize=(10,10))
rows = prediction.shape[0]
for i in range(rows):
  true_mask = mask_from_prob(sample_mask[i])
  pred_mask = mask_from_prob(prediction[i])
  plt.subplot(rows, 3, 3*i +1)
  plt.imshow(sample_image[i], cmap="gray")
  plt.subplot(rows, 3, 3*i + 2)
  plt.imshow(true_mask, cmap="gray")
  plt.subplot(rows, 3, 3*i + 3)
  plt.imshow(pred_mask, cmap="gray")

## Model loading for future use

In [11]:
model = tf.keras.models.load_model(checkpoint_path, custom_objects={'cotp':cotp, 'cott':cott}, compile=False)
weights = [1, 3, 9, 14, 14, 14]     # background, body, liver, kidneys and spleen weights
border_weights = [18] * (33-5)      # organ borders weights
weights = weights + border_weights
model.compile(optimizer = Adam(learning_rate = 1e-4), loss = WeightedCategoricalCrossentropy(weights), metrics = ['accuracy', cotp, cott])   

In [12]:
result = model.evaluate(test_dataset, verbose=2, steps=20)

20/20 - 40s - loss: 0.9038 - accuracy: 0.9672 - cotp: 0.9676 - cott: 0.9666 - 40s/epoch - 2s/step
